In [9]:
# Import packages
import pandas as pd
import psycopg2
import hashlib
import numpy as np

In [41]:
# Import data

Deals = pd.read_csv('data/deal_sample.csv')
Updates = pd.read_csv('data/deal_updates_sample.csv')
Activities = pd.read_csv('data/deal_activities_sample.csv')

#Deals.isnull().values.any()
#Deals['Status'].unique()
#Updates['update_type'].unique()
Activities['Type'].unique()

array(['meeting', 'note', 'soft_claim', 'call', 'task', 'email',
       'comment'], dtype=object)

In [3]:
Deals=Deals.rename(columns={"id":"DealID","user_id":"UserID","Total_activities":"TotalActivities"})
Activities=Activities.rename(columns={"activity_id":"ActivityID","deal_id":"DealID","marked_as_done_ts":"MarkedAsDone","deleted":"Deleted"})
Updates=Updates.rename(columns={"deal_id":"DealID","update_type":"Type","old_value":"Old","new_value":"New"})
Deals = Deals.drop('pipeline_id',1)

C:\Users\mathi\AppData\Local\Temp\ipykernel_10524\4051701644.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  Deals = Deals.drop('pipeline_id',1)


In [37]:
Activities

,activity_id,deal_id,Type,marked_as_done_ts,deleted
0,18165,1580,meeting,2021-04-12,False
1,9436,4250,meeting,2020-12-22,False
2,2537,979,note,2022-01-27,False
3,16188,2867,meeting,2021-12-09,False
4,16402,3127,note,2022-01-13,False
...,...,...,...,...,...
21693,19588,3487,comment,2020-11-26,False
21694,3118,1344,comment,2021-04-15,False
21695,6811,4317,comment,2020-07-20,False
21696,10824,1012,comment,2021-06-03,False


In [19]:



# merge the Deals and Activities dataframes on the "DealID" column
merged_df = pd.merge(Deals, Activities, on="DealID", how="inner")
merged_df = merged_df[(merged_df["Deleted"] == False) & (merged_df["MarkedAsDone"].notnull())]
# group the merged dataframe by "DealID" and count the number of rows in each group
activity_counts = merged_df.groupby("DealID").size()

# create a new column in the Deals dataframe with the activity count for each "DealID"
Deals["ActivityCount"] = Deals["DealID"].map(activity_counts)
Deals = Deals.replace(np.nan,0)
Deals = Deals.astype({"ActivityCount":'int64'})
Deals

,DealID,UserID,Status,Value,Currency,TotalActivities,ActivityCount
0,1,5ws7dNbcfdCZbaaL3OT+R4bUDvFa9Als/7ez7/eswI4=,won,2850.00,SEK,2,2
1,18,5ykOQ/Hwc7V57x8xobyiZTNWG99cTfArh7yLZ2rK1J8=,open,11.55,CZK,0,0
2,46,vGAyegcALaKI2s9xcV3mXacPyY8uZhoM1vc5yy+HetE=,won,1490.00,NOK,6,6
3,51,5ykOQ/Hwc7V57x8xobyiZTNWG99cTfArh7yLZ2rK1J8=,open,125700.00,CLP,0,0
4,73,5ykOQ/Hwc7V57x8xobyiZTNWG99cTfArh7yLZ2rK1J8=,open,1200.00,PLN,4,2
...,...,...,...,...,...,...,...
4995,4690,5L9hFFMl0lXGdvLaI7Yq1W1my1wcL/uWx2ZeRAvoYOs=,won,1300.00,DKK,4,4
4996,4770,vGAyegcALaKI2s9xcV3mXacPyY8uZhoM1vc5yy+HetE=,lost,3725.00,NOK,0,0
4997,4821,KEwpw1mo7NXYT5Q8SgkUgdcfIOQCs+SHOWQVPrdVaxc=,open,6500.00,NOK,7,7
4998,4824,5ykOQ/Hwc7V57x8xobyiZTNWG99cTfArh7yLZ2rK1J8=,lost,3850.00,NOK,1,1


In [20]:
Accuracy = 0
for _,i in Deals.iterrows():
    if i['TotalActivities'] == i['ActivityCount']:
        Accuracy = Accuracy + 1

print(Accuracy/5000)

0.9378


In [24]:
# merge the Deals and Activities dataframes on the "DealID" column
merged_df = pd.merge(Deals, Updates, on="DealID", how="inner")
merged_df = merged_df[(merged_df["Type"] == "value") | (merged_df["Type"] == "stage_id")| (merged_df["Type"] == "status")]
# group the merged dataframe by "DealID" and count the number of rows in each group
activity_counts = merged_df.groupby("DealID").size()

# create a new column in the Deals dataframe with the activity count for each "DealID"
Deals["UpdateCount"] = Deals["DealID"].map(activity_counts)#
Deals = Deals.replace(np.nan,0)
Deals = Deals.astype({"UpdateCount":'int64'})
np.average(Deals['UpdateCount'])

4.9182

In [27]:
Deals['Active']= np.where((Deals['ActivityCount']==0) & (Deals['UpdateCount']==0),'Inactive','Active')
Deals

,DealID,UserID,Status,Value,Currency,TotalActivities,ActivityCount,UpdateCount,Active
0,1,5ws7dNbcfdCZbaaL3OT+R4bUDvFa9Als/7ez7/eswI4=,won,2850.00,SEK,2,2,5,Active
1,18,5ykOQ/Hwc7V57x8xobyiZTNWG99cTfArh7yLZ2rK1J8=,open,11.55,CZK,0,0,0,Inactive
2,46,vGAyegcALaKI2s9xcV3mXacPyY8uZhoM1vc5yy+HetE=,won,1490.00,NOK,6,6,5,Active
3,51,5ykOQ/Hwc7V57x8xobyiZTNWG99cTfArh7yLZ2rK1J8=,open,125700.00,CLP,0,0,0,Inactive
4,73,5ykOQ/Hwc7V57x8xobyiZTNWG99cTfArh7yLZ2rK1J8=,open,1200.00,PLN,4,2,2,Active
...,...,...,...,...,...,...,...,...,...
4995,4690,5L9hFFMl0lXGdvLaI7Yq1W1my1wcL/uWx2ZeRAvoYOs=,won,1300.00,DKK,4,4,9,Active
4996,4770,vGAyegcALaKI2s9xcV3mXacPyY8uZhoM1vc5yy+HetE=,lost,3725.00,NOK,0,0,2,Active
4997,4821,KEwpw1mo7NXYT5Q8SgkUgdcfIOQCs+SHOWQVPrdVaxc=,open,6500.00,NOK,7,7,5,Active
4998,4824,5ykOQ/Hwc7V57x8xobyiZTNWG99cTfArh7yLZ2rK1J8=,lost,3850.00,NOK,1,1,1,Active


In [29]:
# merge the Deals and Activities dataframes on the "DealID" column
merged_df_2 = pd.merge(Deals, Activities, on="DealID", how="inner")
merged_df_2 = merged_df_2[(merged_df_2["Type"]=='email')&(merged_df_2['Status']=='won')]
# group the merged dataframe by "DealID" and count the number of rows in each group
activity_counts1 = merged_df_2.groupby("DealID").size()
Deals["EmailCount"] = Deals["DealID"].map(activity_counts1)
# merge the Deals and Activities dataframes on the "DealID" column
merged_df_3 = pd.merge(Deals, Activities, on="DealID", how="inner")
merged_df_3 = merged_df_3[(merged_df_3["Type"]=='call')&(merged_df_3['Status']=='won')]
# group the merged dataframe by "DealID" and count the number of rows in each group
activity_counts2 = merged_df_3.groupby("DealID").size()


# create a new column in the Deals dataframe with the activity count for each "DealID"
Deals["CallCount"] = Deals["DealID"].map(activity_counts2)
Deals = Deals.replace(np.nan,0)
Deals = Deals.astype({"EmailCount":'int64',"CallCount":"int64"})
WonDeals = Deals[Deals['Status']=='won']
WonDeals

,DealID,UserID,Status,Value,Currency,TotalActivities,ActivityCount,UpdateCount,Active,EmailCount,CallCount
0,1,5ws7dNbcfdCZbaaL3OT+R4bUDvFa9Als/7ez7/eswI4=,won,2850.0,SEK,2,2,5,Active,0,0
2,46,vGAyegcALaKI2s9xcV3mXacPyY8uZhoM1vc5yy+HetE=,won,1490.0,NOK,6,6,5,Active,0,3
6,227,nlOu78vH4eXtXC/20kcZwLoGAnfgOxVsP5EAPrsCSvU=,won,950.0,SEK,6,6,4,Active,4,0
11,358,d9Rhf2pUR14q6zNjQdR+wdqeAqwnFFZtqDkk/Yqr2S0=,won,950.0,SEK,5,5,6,Active,0,0
15,445,HBiOphMW6jnNyJPyZoR74FpR/7yYcM+96TZ5NGNH7EM=,won,7800.0,NOK,9,9,4,Active,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4985,3871,5Y6Xi1DHdxA6ymrCffP0dYqcmE5wHZ5t424iln15ehU=,won,2850.0,SEK,2,2,3,Active,0,1
4989,4043,5Y6Xi1DHdxA6ymrCffP0dYqcmE5wHZ5t424iln15ehU=,won,5100.0,SEK,3,3,4,Active,0,0
4990,4161,Tj9pqthLqOsYCO8GMVdRlOjSWLloT7lNcSrO9d+w8ew=,won,3437.5,DKK,2,2,6,Active,0,1
4995,4690,5L9hFFMl0lXGdvLaI7Yq1W1my1wcL/uWx2ZeRAvoYOs=,won,1300.0,DKK,4,4,9,Active,0,0


In [15]:
#look for rows with nulls then analyse those rows (should we keep or drop depending on what is null)
#Deals has no nulls, Activities has no null besides the date column which is normal because that means its not closed, Updates has null values in the old and new value columns
#for the old value column it makes sense since a deal starts with no person or organisation data and more gets added on. But for the new value does it makes sense to keep those rows ? Not 100% 
#sure because only affects 11 rows but if we delete that data then we wont be able to recreate the history of updates of a specific deal because we would miss a step. Therefore in my opinion
#the rows with empty new values should be kept for the integrity of the data in case we would want to create a timeline of updates per deal
emptyupdate = Updates[Updates['new_value'].isnull()]
emptyupdate

,deal_id,update_type,old_value,new_value
25104,4020,org_id,20113,NaN
25115,2960,org_id,20454,NaN
25172,3726,org_id,21472,NaN
25179,868,org_id,21624,NaN
25180,2515,org_id,21625,NaN
25189,1968,org_id,21928,NaN
25535,2576,org_id,30324,NaN
25625,3570,org_id,32244,NaN
25634,3615,org_id,32438,NaN
25673,4795,org_id,33723,NaN


In [53]:
#Writing script to create the insert into statement and appending it to the create_statements file

def auto_insert(dataframe, table):
    col_names = list(dataframe.columns)
    with open('create_statements.sql','a+') as outfile:
        for i in dataframe.itertuples(index=False):
            val = [str(value) if pd.notnull(value) else "NULL" for value in i]
            insert=(f"INSERT INTO {table} ({','.join(col_names)}) VALUES ({','.join(val)});\n")
            outfile.write(insert)

In [54]:
auto_insert(Deals,"Deals")
auto_insert(Activities,"Activities")
auto_insert(Updates,"Updates")

In [ ]:
#Check if total activities field is accurate
#only count activitity if Markedasdone is not empty and deleted is False


#Find avg numbers of updates per deal only count stages values and status
# if deal has num of updates = 0 and done activities = 0 too then account = inactive

#Number of calls and email for each deal won 

#create view where deal has no activity or no activity within 2 years


In [ ]:
# Transform

In [57]:
int('5ws7dNbcfdCZbaaL3OT+R4bUDvFa9Als/7ez7/eswI4=',36)

ValueError: invalid literal for int() with base 36: '5ws7dNbcfdCZbaaL3OT+R4bUDvFa9Als/7ez7/eswI4='